# Predicting the winner of the 2018 FIFA World Cup
**Aim**: 
to predict the outcome of the FIFA World Cup 2018

 **Method**:
- Use data from Kaggle to model the outcome of certain pairings between teams, given their rank, points, and the weighted point difference with the opponent.
- Use this model to predict the outcome of the group rounds and then the single-elimination phase

**Summary**
According to the analysis below, the team that will win the World Cup is the [underdog according to the bookies](https://www.oddschecker.com/football/world-cup/winner): Germany . But this is not the only interesting thing you can observe. If [not interested in the data munging part](https://ih1.redbubble.net/image.441592463.7099/sticker,375x360-bg,ffffff.u1.png) head to the last section (Visualization) and see the single-elimination phase. To make the story more interesting for the sport lovers, I report fair odds (i.e. fair market, without bookmaker margin) instead of the raw probability output. 

Couple things to note, 
- it seems Egypt qualifies to the best 16 and their match with Portugal is the safest bet in the phase. 
- In this simulation, Brasil cannot make it through Belgium.. I'm sure here many would consider otherwise
- It seems that neither CRonaldo (POR) nor LMessi (ARG) is going to play in the final but they are going to play against each other in the quarterfinals...

All in all, I'm looking forward to the tournament! Place your bets and get ready you too! 

Cheers!

# Data I/O

I use three datasets
- FIFA rankings from 1993 to 2018 (courtesy of [Tadhg Fitzgerald](https://www.kaggle.com/tadhgfitzgerald) 
> This one I used to get the FIFA ranking and points for the teams, which is a monthly changing rank previously shown as a [decent predictor](http://www.sportingintelligence.com/2014/06/20/upsets-giant-killings-adios-bye-bye-fifa-rankings-still-ahead-in-predicting-results-200601/) of team performance
- International Soccer matches from 1872 to 2018 (courtesy of [Mart Jürisoo](https://www.kaggle.com/martj42))
> This I will use to find out how much the difference in point, ranks and the current rank of the team affects the outocme of a match
- FIFA World Cup 2018 data set (courtesy of [Nuggs](https://www.kaggle.com/ahmedelnaggar))
> This I will use to get the upcoming matches 

Comment: Right when reading the datafiles I take care of the renaming of the Countries because there is some mismatch in the sources

In [38]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from matplotlib import pyplot as plt
pd.options.display.max_columns = 999


rankings = pd.read_csv('/var/sds/homes/A127417/workspace/fifa_ranking.csv')
rankings = rankings.loc[:,['rank', 'country_full', 'country_abrv', 'cur_year_avg_weighted', 'rank_date', 
                           'two_year_ago_weighted', 'three_year_ago_weighted']]
rankings = rankings.replace({"IR Iran": "Iran"})
rankings['weighted_points'] =  rankings['cur_year_avg_weighted'] + rankings['two_year_ago_weighted'] + rankings['three_year_ago_weighted']
rankings['rank_date'] = pd.to_datetime(rankings['rank_date'])

matches = pd.read_csv('/var/sds/homes/A127417/workspace/results.csv')
matches =  matches.replace({'Germany DR': 'Germany', 'China': 'China PR'})
matches['date'] = pd.to_datetime(matches['date'])
matches['year'] = matches['date'].dt.year


world_cup = pd.read_csv('/var/sds/homes/A127417/workspace/World_Cup_2018_Dataset.csv')
world_cup = world_cup.loc[:, ['Team', 'Group', 'First match \nagainst', 'Second match\n against', 'Third match\n against']]
world_cup = world_cup.dropna(how='all')
world_cup = world_cup.replace({"IRAN": "Iran", 
                               "Costarica": "Costa Rica", 
                               "Porugal": "Portugal", 
                               "Columbia": "Colombia", 
                               "Korea" : "Korea Republic"})
world_cup = world_cup.set_index('Team')

elo_ratings = pd.read_csv('/var/sds/homes/A127417/workspace/Final_World-Football-Elo-Ratings.csv')
elo_ratings['Win_pct']=elo_ratings['Win']/elo_ratings['Total']
elo_ratings['Losses_pct']=elo_ratings['Losses']/elo_ratings['Total']
elo_ratings['Draws_pct']=elo_ratings['Draws']/elo_ratings['Total']
elo_ratings = elo_ratings.replace({"South Korea" : "Korea Republic"})
elo_ratings = elo_ratings.loc[:,['Year','Global','Team','Rating','Avg Rank','Avg Rating','Year Rank', 'Year Rating','Total','Win_pct','Losses_pct','Draws_pct']]

bets = pd.read_csv('/var/sds/homes/A127417/workspace/bets.csv')
bets
#elo_ratings['Year Rating'] = pd.to_numeric(elo_ratings['Year Rating'])#.astype(int)
#elo_ratings
#elo_ratings.dtypes
#matches2 = matches[matches.isnull().any(axis=1)]
#matches


,Team,Prob
0,Brazil,4.0
1,Argentina,9.0
2,England,16.0
3,Spain,6.5
4,Germany,4.5
5,Uruguay,28.0
6,Croatia,33.0
7,Sweden,150.0
8,Denmark,100.0
9,Serbia,150.0


# Feature extraction

I join the matches with the ranks of the different teams. 

Then extract some features:
- point and rank differences
- if the game was for some stakes, because my naive view was that typically friendly matches are harder to predict (TODO differentiate the WC matches from the rest)
- how many days the different teams were able to rest but this turned out to be not important enough to be worth the hassle
- include the participant countries as a one hot vector but that did not appear to be a strong predictor either

In [39]:
# I want to have the ranks for every day 
rankings = rankings.set_index(['rank_date'])\
            .groupby(['country_full'], group_keys=False)\
            .resample('D').first()\
            .fillna(method='ffill')\
            .reset_index()

# join the ranks
matches = matches.merge(rankings, 
                        left_on=['date', 'home_team'], 
                        right_on=['rank_date', 'country_full'])

matches = matches.merge(rankings, 
                        left_on=['date', 'away_team'], 
                        right_on=['rank_date', 'country_full'], 
                        suffixes=('_home', '_away'))

matches = matches.merge(elo_ratings, 
                        left_on=['year', 'home_team'], 
                        right_on=['Year', 'Team'])

matches = matches.merge(elo_ratings, 
                        left_on=['year', 'away_team'], 
                        right_on=['Year', 'Team'], 
                       suffixes=('_home', '_away'))


#matches.loc[matches['year'] == 2018]



In [40]:
# let's define the rankings at the time of the World Cup
world_cup_rankings = rankings.loc[(rankings['rank_date'] == rankings['rank_date'].max()) & 
                                    rankings['country_full'].isin(world_cup.index.unique())]
world_cup_rankings['year'] = 2018
#world_cup_rankings.dtypes
#elo_ratings.dtypes
world_cup_rankings = world_cup_rankings.merge(elo_ratings, 
                        left_on=['year', 'country_full'], 
                        right_on=['Year', 'Team'])
world_cup_rankings = world_cup_rankings.merge(bets, 
                        left_on=['country_full'], 
                        right_on=['Team'])
world_cup_rankings = world_cup_rankings.set_index(['country_full'])

world_cup_rankings
# world_cup_rankings = world_cup_rankings.set_index(['Team'])
# ver, me convierte el country_full en un indice cuando hago el merge

#world_cup_rankings
#print(rankings)
#print(matches)
#print(world_cup)

/opt/conda/lib/python3.5/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.


,rank_date,rank,country_abrv,cur_year_avg_weighted,two_year_ago_weighted,three_year_ago_weighted,weighted_points,year,Year,Global,Team_x,Rating,Avg Rank,Avg Rating,Year Rank,Year Rating,Total,Win_pct,Losses_pct,Draws_pct,Team_y,Prob
country_full,,,,,,,,,,,,,,,,,,,,,,
Argentina,2018-06-07,5.0,ARG,404.07,248.99,183.59,836.65,2018,2018,5,Argentina,1986,5,1983,-2,-32,1004,0.534861,0.217131,0.248008,Argentina,9.0
Australia,2018-06-07,36.0,AUS,366.60,98.16,59.79,524.55,2018,2018,32,Australia,1742,36,1650,3,22,537,0.506518,0.279330,0.214153,Australia,300.0
Belgium,2018-06-07,3.0,BEL,629.98,158.94,186.58,975.50,2018,2018,8,Belgium,1939,24,1735,4,40,771,0.418936,0.367056,0.214008,Belgium,11.0
Brazil,2018-06-07,2.0,BRA,558.95,168.06,162.38,889.39,2018,2018,1,Brazil,2142,4,1994,0,32,964,0.633817,0.162863,0.203320,Brazil,4.0
Colombia,2018-06-07,16.0,COL,292.09,199.73,166.38,658.20,2018,2018,9,Colombia,1928,51,1586,-1,-36,556,0.388489,0.341727,0.269784,Colombia,40.0
Costa Rica,2018-06-07,23.0,CRC,410.90,99.73,133.11,643.74,2018,2018,31,Costa Rica,1744,34,1695,-1,-14,683,0.465593,0.297218,0.237189,Costa Rica,500.0
Croatia,2018-06-07,20.0,CRO,397.75,100.79,110.25,608.79,2018,2018,17,Croatia,1853,12,1873,-1,2,299,0.531773,0.200669,0.267559,Croatia,33.0
Denmark,2018-06-07,12.0,DEN,723.72,90.07,80.21,894.00,2018,2018,16,Denmark,1856,20,1794,18,117,806,0.456576,0.341191,0.202233,Denmark,100.0
Egypt,2018-06-07,45.0,EGY,208.27,99.48,50.64,358.39,2018,2018,50,Egypt,1646,34,1672,-2,-20,755,0.504636,0.256954,0.238411,Egypt,200.0


In [41]:
# feature generation
matches['rank_difference'] = matches['rank_home'] - matches['rank_away']
matches['point_difference'] = matches['weighted_points_home'] - matches['weighted_points_away']
matches['score_difference'] = matches['home_score'] - matches['away_score']
matches['elo_difference'] = matches['Rating_home'] - matches['Rating_away']
matches['winptc_difference'] = matches['Win_pct_home'] - matches['Win_pct_away']
matches['lossptc_difference'] = matches['Losses_pct_home'] - matches['Losses_pct_away']
matches['avgrank_difference'] = matches['Avg Rank_home'] - matches['Avg Rank_away']
matches['avgrating_difference'] = matches['Avg Rating_home'] - matches['Avg Rating_away']
matches['yearrating_difference'] = matches['Year Rating_home'] - matches['Year Rating_away']
matches['is_won'] = matches['score_difference'] > 0 # take draw as lost
matches['is_stake'] = matches['tournament'] != 'Friendly'


#matches2 = matches[matches.isnull().any(axis=1)]
#matches

# I tried earlier rest days but it did not turn to be useful
#max_rest = 30
#matches['rest_days'] = matches.groupby('home_team').diff()['date'].dt.days.clip(0,max_rest).fillna(max_rest)

# I tried earlier the team as well but that did not make a difference either
#matches['wc_participant'] = matches['home_team'] * matches['home_team'].isin(world_cup.index.tolist())
#matches['wc_participant'] = matches['wc_participant'].replace({'':'Other'})
#matches = matches.join(pd.get_dummies(matches['wc_participant']))

# Modeling



In [46]:
########
###### Gradient Boosting #########
from numpy import loadtxt
from sklearn.metrics import accuracy_score
from sklearn import model_selection
from sklearn.ensemble import GradientBoostingClassifier
from sklearn import preprocessing

X, y = matches.loc[:,['rank_home', 'rank_difference', 'point_difference', 'is_stake'
                      ,'Rating_home','elo_difference']], matches['is_won'] 
                                            #,'winptc_difference','yearrating_difference','avgrating_difference''Win_pct_home','Losses_pct_home','Draws_pct_home'
#print(X)                     
# Se escalan las variables en una variable reutilizable para despues aplicarla a la predicción
#scaler = preprocessing.StandardScaler().fit(X)
#X_scaled = scaler.transform(X)
#X_scaled = pd.DataFrame(X_scaled, columns=['rank_home', 'rank_difference', 'point_difference', 'is_stake'
                      #,'Rating_home','elo_difference','winptc_difference','yearrating_difference'])
#X['yearrating_difference'] = X_scaled['yearrating_difference']

print(X.columns) #['yearrating_difference']
seed = 7
num_trees = 50
kfold = model_selection.KFold(n_splits=10, random_state=seed)
params = {'n_estimators': num_trees, 'max_depth': 3, 'learning_rate': 0.05}
model = GradientBoostingClassifier(**params)
#model = GradientBoostingClassifier(n_estimators=num_trees, random_state=seed)
model.fit(X,y)
model_acc = model_selection.cross_val_score(model, X, y, cv=kfold)
#print(model.feature_importances_)

#print(X['yearrating_difference'].reshape(-1, 1))
#print(model.predict_proba(X))
#model = model.fit(X,y)


# Accuracy con GB / Trees 100 / max_depth 3 , LR = 0.1 / Cv 10 = 0.6725
# Accuracy con GB + ELO + winptc/ Trees 100 / max_depth 3 , LR = 0.05 / Cv 10 = 0.7025988
# Accuracy con GB + ELO + winptc + avgrating + yearrating/ Trees 50 / max_depth 3 , LR = 0.05 / Cv 10 = 0.737966



########
###### RANDOM FOREST #########
#from numpy import loadtxt
#from sklearn.metrics import accuracy_score
#from sklearn import model_selection
#from sklearn.ensemble import RandomForestClassifier

#X, y = matches.loc[:,['rank_home', 'rank_difference', 'point_difference', 'is_stake']], matches['is_won']


#seed = 7
#num_trees = 100
#max_features = 3
#kfold = model_selection.KFold(n_splits=10, random_state=seed)
#model = RandomForestClassifier(n_estimators=num_trees, max_features=max_features)
#results = model_selection.cross_val_score(model, X, y, cv=kfold)
#print(results.mean())


#from numpy import loadtxt
#from sklearn.metrics import accuracy_score
#from sklearn import model_selection
#from sklearn.ensemble import BaggingClassifier
#from sklearn.ensemble import GradientBoostingClassifier

###### BAGGING #########
#X, y = matches.loc[:,['rank_home', 'rank_difference', 'point_difference', 'is_stake']], matches['is_won']
#seed = 7
#kfold = model_selection.KFold(n_splits=10, random_state=seed)
#cart = DecisionTreeClassifier()
#num_trees = 500
#model = BaggingClassifier(base_estimator=cart, n_estimators=num_trees, random_state=seed)
#results = model_selection.cross_val_score(model, X, y, cv=kfold)
#print(results)
#Accuracy con muestra 20% / 100 arboles = 0.62373

Index(['rank_home', 'rank_difference', 'point_difference', 'is_stake',
       'Rating_home', 'elo_difference'],
      dtype='object')


I consider this pretty good performance, minding that soccer matches have typically only few goals scored and therefore making their outcome even more unpredictable. Nevertheless, let's look at the bad predictions and see where we are making mistakes more often.

# World Cup simulation

## Group rounds

In [49]:
import random
import operator
import enum 

class Rondas(enum.IntEnum):
     Grupo = 1
     Octavos = 2
     Cuartos = 3
     Semifinal = 4
     TercerPuesto = 5
     Final = 6
     Campeon = 7

class Equipo(object):
    def __init__(self,nombre,palpito=1):
        self.nombre = nombre
        self.ranking = random.random()
        self.palpito=palpito
    def __str__(self):
        return str(self.nombre)

        
class TablaRegistro(object):
    def __init__(self):
        self.puntos = 0
        self.golesAFavor = 0
        self.golesEnContra = 0
        self.fase = Rondas.Grupo
    def SetPuntos(self,puntos):
        self.puntos = puntos
    def __str__(self):
        return str(self.fase) + ' ' + str(self.puntos) + '-' + str(self.golesAFavor) + '-' + str(self.golesEnContra)
        

class Resultado(object):
    def __init__(self,golesA,golesB):
        self.golesA = golesA
        self.golesB = golesB
    def __str__(self):
        return str(self.golesA) + ' ' + str(self.golesB)         

class Tabla(object):
    def __init__(self,equipos):

        nombres = [o.nombre for o in equipos]
        self.tabla = dict()
        for i in nombres:
            self.tabla[i] = TablaRegistro()
    def setFase(self,equipo,fase):
            self.tabla[equipo.nombre].fase = fase
            
    def resultado(self,equipo1,equipo2,goles1,goles2):
        if goles1==goles2:
            self.tabla[equipo1.nombre].puntos = self.tabla[equipo1.nombre].puntos + 1
            self.tabla[equipo1.nombre].golesAFavor = self.tabla[equipo1.nombre].golesAFavor + goles1
            self.tabla[equipo1.nombre].golesEnContra = self.tabla[equipo1.nombre].golesEnContra + goles2
            self.tabla[equipo2.nombre].puntos = self.tabla[equipo2.nombre].puntos + 1
            self.tabla[equipo2.nombre].golesAFavor = self.tabla[equipo2.nombre].golesAFavor + goles2
            self.tabla[equipo2.nombre].golesEnContra = self.tabla[equipo2.nombre].golesEnContra + goles1
        elif goles1>goles2:
            self.tabla[equipo1.nombre].puntos = self.tabla[equipo1.nombre].puntos + 3
            self.tabla[equipo1.nombre].golesAFavor = self.tabla[equipo1.nombre].golesAFavor + goles1
            self.tabla[equipo1.nombre].golesEnContra = self.tabla[equipo1.nombre].golesEnContra + goles2
            self.tabla[equipo2.nombre].golesAFavor = self.tabla[equipo2.nombre].golesAFavor + goles2
            self.tabla[equipo2.nombre].golesEnContra = self.tabla[equipo2.nombre].golesEnContra + goles1
        else:
            self.tabla[equipo1.nombre].golesAFavor = self.tabla[equipo1.nombre].golesAFavor + goles1
            self.tabla[equipo1.nombre].golesEnContra = self.tabla[equipo1.nombre].golesEnContra + goles2
            self.tabla[equipo2.nombre].puntos = self.tabla[equipo2.nombre].puntos + 3
            self.tabla[equipo2.nombre].golesAFavor = self.tabla[equipo2.nombre].golesAFavor + goles2
            self.tabla[equipo2.nombre].golesEnContra = self.tabla[equipo2.nombre].golesEnContra + goles1

    def getEquiposOrden(self):
#TODO: ordenar de acuerdo a las reglas de la FIFA
        sorted_d = sorted(self.tabla, key = lambda name: (self.tabla[name].fase, self.tabla[name].puntos), reverse=True)
        return sorted_d
            
    def __str__(self):
        c_list = 'Tabla\n' + '--------P-Gf-GC---------\n'
        equiposOrden = sorted(self.tabla, key = lambda name: (self.tabla[name].fase,self.tabla[name].puntos), reverse=True)
        for key in equiposOrden:
            c_list += str(key) + ' ' + str(self.tabla[key]) + '\n' 
        return c_list

class Grupo(object):
    def __init__(self,equipo1,equipo2,equipo3,equipo4):
        self.equipo1 = equipo1
        self.equipo2 = equipo2
        self.equipo3 = equipo3
        self.equipo4 = equipo4
        self.tabla =  Tabla([equipo1,equipo2,equipo3,equipo4])
 
    def equipos(self):
        return [self.equipo1,self.equipo2,self.equipo3,self.equipo4]
    def clasificados(self):
        primero, segundo = self.tabla.getEquiposOrden()[0:2]
        clasificadosLista= []
        if self.equipo1.nombre == primero:
           clasificadosLista.append(self.equipo1)
        if self.equipo2.nombre == primero:
           clasificadosLista.append(self.equipo2)
        if self.equipo3.nombre == primero:
           clasificadosLista.append(self.equipo3)
        if self.equipo4.nombre == primero:
           clasificadosLista.append(self.equipo4)

        if self.equipo1.nombre == segundo:
           clasificadosLista.append(self.equipo1)
        if self.equipo2.nombre == segundo:
           clasificadosLista.append(self.equipo2)
        if self.equipo3.nombre == segundo:
           clasificadosLista.append(self.equipo3)
        if self.equipo4.nombre == segundo:
           clasificadosLista.append(self.equipo4)
        return clasificadosLista
        
class Mundial(object):
    def __init__(self):
        self.GrupoA = Grupo(Equipo('Russia'),Equipo('Saudi Arabia'),Equipo('Egypt'),Equipo('Uruguay'))
        self.GrupoB = Grupo(Equipo('Portugal'),Equipo('Spain'),Equipo('Morocco'),Equipo('Iran'))
        self.GrupoC = Grupo(Equipo('France'),Equipo('Australia'),Equipo('Peru'),Equipo('Denmark'))
        self.GrupoD = Grupo(Equipo('Argentina'),Equipo('Iceland'),Equipo('Croatia'),Equipo('Nigeria'))
        self.GrupoE = Grupo(Equipo('Brazil'),Equipo('Switzerland'),Equipo('Costa Rica'),Equipo('Serbia'))
        self.GrupoF = Grupo(Equipo('Germany'),Equipo('Mexico'),Equipo('Sweden'),Equipo('Korea Republic'))
        self.GrupoG = Grupo(Equipo('Belgium'),Equipo('Panama'),Equipo('Tunisia'),Equipo('England'))
        self.GrupoH = Grupo(Equipo('Poland'),Equipo('Senegal'),Equipo('Colombia'),Equipo('Japan'))
        self.tabla =  Tabla(self.GrupoA.equipos()+self.GrupoB.equipos()+self.GrupoC.equipos()+self.GrupoD.equipos()+self.GrupoE.equipos()+self.GrupoF.equipos()+self.GrupoG.equipos()+self.GrupoH.equipos())

    def jugarFechaGrupo(self,numero):
        if numero==1:
             resultado = self.partido(self.GrupoA.equipo1,self.GrupoA.equipo2)
             self.tabla.resultado(self.GrupoA.equipo1,self.GrupoA.equipo2,resultado.golesA,resultado.golesB)
             self.GrupoA.tabla.resultado(self.GrupoA.equipo1,self.GrupoA.equipo2,resultado.golesA,resultado.golesB)

             resultado = self.partido(self.GrupoB.equipo1,self.GrupoB.equipo2)
             self.tabla.resultado(self.GrupoB.equipo1,self.GrupoB.equipo2,resultado.golesA,resultado.golesB)
             self.GrupoB.tabla.resultado(self.GrupoB.equipo1,self.GrupoB.equipo2,resultado.golesA,resultado.golesB)

             resultado = self.partido(self.GrupoC.equipo1,self.GrupoC.equipo2)
             self.tabla.resultado(self.GrupoC.equipo1,self.GrupoC.equipo2,resultado.golesA,resultado.golesB)
             self.GrupoC.tabla.resultado(self.GrupoC.equipo1,self.GrupoC.equipo2,resultado.golesA,resultado.golesB)

             resultado = self.partido(self.GrupoD.equipo1,self.GrupoD.equipo2)
             self.tabla.resultado(self.GrupoD.equipo1,self.GrupoD.equipo2,resultado.golesA,resultado.golesB)
             self.GrupoD.tabla.resultado(self.GrupoD.equipo1,self.GrupoD.equipo2,resultado.golesA,resultado.golesB)
             
             resultado = self.partido(self.GrupoE.equipo1,self.GrupoE.equipo2)
             self.tabla.resultado(self.GrupoE.equipo1,self.GrupoE.equipo2,resultado.golesA,resultado.golesB)
             self.GrupoE.tabla.resultado(self.GrupoE.equipo1,self.GrupoE.equipo2,resultado.golesA,resultado.golesB)

             resultado = self.partido(self.GrupoF.equipo1,self.GrupoF.equipo2)
             self.tabla.resultado(self.GrupoF.equipo1,self.GrupoF.equipo2,resultado.golesA,resultado.golesB)
             self.GrupoF.tabla.resultado(self.GrupoF.equipo1,self.GrupoF.equipo2,resultado.golesA,resultado.golesB)
             
             resultado = self.partido(self.GrupoG.equipo1,self.GrupoG.equipo2)
             self.tabla.resultado(self.GrupoG.equipo1,self.GrupoG.equipo2,resultado.golesA,resultado.golesB)
             self.GrupoG.tabla.resultado(self.GrupoG.equipo1,self.GrupoG.equipo2,resultado.golesA,resultado.golesB)

             resultado = self.partido(self.GrupoH.equipo1,self.GrupoH.equipo2)
             self.tabla.resultado(self.GrupoH.equipo1,self.GrupoH.equipo2,resultado.golesA,resultado.golesB)
             self.GrupoH.tabla.resultado(self.GrupoH.equipo1,self.GrupoH.equipo2,resultado.golesA,resultado.golesB)
             ###################
             resultado = self.partido(self.GrupoA.equipo3,self.GrupoA.equipo4)
             self.tabla.resultado(self.GrupoA.equipo3,self.GrupoA.equipo4,resultado.golesA,resultado.golesB)
             self.GrupoA.tabla.resultado(self.GrupoA.equipo3,self.GrupoA.equipo4,resultado.golesA,resultado.golesB)

             resultado = self.partido(self.GrupoB.equipo3,self.GrupoB.equipo4)
             self.tabla.resultado(self.GrupoB.equipo3,self.GrupoB.equipo4,resultado.golesA,resultado.golesB)
             self.GrupoB.tabla.resultado(self.GrupoB.equipo3,self.GrupoB.equipo4,resultado.golesA,resultado.golesB)

             resultado = self.partido(self.GrupoC.equipo3,self.GrupoC.equipo4)
             self.tabla.resultado(self.GrupoC.equipo3,self.GrupoC.equipo4,resultado.golesA,resultado.golesB)
             self.GrupoC.tabla.resultado(self.GrupoC.equipo3,self.GrupoC.equipo4,resultado.golesA,resultado.golesB)

             resultado = self.partido(self.GrupoD.equipo3,self.GrupoD.equipo4)
             self.tabla.resultado(self.GrupoD.equipo3,self.GrupoD.equipo4,resultado.golesA,resultado.golesB)
             self.GrupoD.tabla.resultado(self.GrupoD.equipo3,self.GrupoD.equipo4,resultado.golesA,resultado.golesB)
             
             resultado = self.partido(self.GrupoE.equipo3,self.GrupoE.equipo4)
             self.tabla.resultado(self.GrupoE.equipo3,self.GrupoE.equipo4,resultado.golesA,resultado.golesB)
             self.GrupoE.tabla.resultado(self.GrupoE.equipo3,self.GrupoE.equipo4,resultado.golesA,resultado.golesB)

             resultado = self.partido(self.GrupoF.equipo3,self.GrupoF.equipo4)
             self.tabla.resultado(self.GrupoF.equipo3,self.GrupoF.equipo4,resultado.golesA,resultado.golesB)
             self.GrupoF.tabla.resultado(self.GrupoF.equipo3,self.GrupoF.equipo4,resultado.golesA,resultado.golesB)
             
             resultado = self.partido(self.GrupoG.equipo3,self.GrupoG.equipo4)
             self.tabla.resultado(self.GrupoG.equipo3,self.GrupoG.equipo4,resultado.golesA,resultado.golesB)
             self.GrupoG.tabla.resultado(self.GrupoG.equipo3,self.GrupoG.equipo4,resultado.golesA,resultado.golesB)

             resultado = self.partido(self.GrupoH.equipo3,self.GrupoH.equipo4)
             self.tabla.resultado(self.GrupoH.equipo3,self.GrupoH.equipo4,resultado.golesA,resultado.golesB)
             self.GrupoH.tabla.resultado(self.GrupoH.equipo3,self.GrupoH.equipo4,resultado.golesA,resultado.golesB)

             
             
        if numero==2:
             resultado = self.partido(self.GrupoA.equipo1,self.GrupoA.equipo3)
             self.tabla.resultado(self.GrupoA.equipo1,self.GrupoA.equipo3,resultado.golesA,resultado.golesB)
             self.GrupoA.tabla.resultado(self.GrupoA.equipo1,self.GrupoA.equipo3,resultado.golesA,resultado.golesB)

             resultado = self.partido(self.GrupoB.equipo1,self.GrupoB.equipo3)
             self.tabla.resultado(self.GrupoB.equipo1,self.GrupoB.equipo3,resultado.golesA,resultado.golesB)
             self.GrupoB.tabla.resultado(self.GrupoB.equipo1,self.GrupoB.equipo3,resultado.golesA,resultado.golesB)

             resultado = self.partido(self.GrupoC.equipo1,self.GrupoC.equipo3)
             self.tabla.resultado(self.GrupoC.equipo1,self.GrupoC.equipo3,resultado.golesA,resultado.golesB)
             self.GrupoC.tabla.resultado(self.GrupoC.equipo1,self.GrupoC.equipo3,resultado.golesA,resultado.golesB)

             resultado = self.partido(self.GrupoD.equipo1,self.GrupoD.equipo3)
             self.tabla.resultado(self.GrupoD.equipo1,self.GrupoD.equipo3,resultado.golesA,resultado.golesB)
             self.GrupoD.tabla.resultado(self.GrupoD.equipo1,self.GrupoD.equipo3,resultado.golesA,resultado.golesB)
             
             resultado = self.partido(self.GrupoE.equipo1,self.GrupoE.equipo3)
             self.tabla.resultado(self.GrupoE.equipo1,self.GrupoE.equipo3,resultado.golesA,resultado.golesB)
             self.GrupoE.tabla.resultado(self.GrupoE.equipo1,self.GrupoE.equipo3,resultado.golesA,resultado.golesB)

             resultado = self.partido(self.GrupoF.equipo1,self.GrupoF.equipo3)
             self.tabla.resultado(self.GrupoF.equipo1,self.GrupoF.equipo3,resultado.golesA,resultado.golesB)
             self.GrupoF.tabla.resultado(self.GrupoF.equipo1,self.GrupoF.equipo3,resultado.golesA,resultado.golesB)
             
             resultado = self.partido(self.GrupoG.equipo1,self.GrupoG.equipo3)
             self.tabla.resultado(self.GrupoG.equipo1,self.GrupoG.equipo3,resultado.golesA,resultado.golesB)
             self.GrupoG.tabla.resultado(self.GrupoG.equipo1,self.GrupoG.equipo3,resultado.golesA,resultado.golesB)

             resultado = self.partido(self.GrupoH.equipo1,self.GrupoH.equipo3)
             self.tabla.resultado(self.GrupoH.equipo1,self.GrupoH.equipo3,resultado.golesA,resultado.golesB)
             self.GrupoH.tabla.resultado(self.GrupoH.equipo1,self.GrupoH.equipo3,resultado.golesA,resultado.golesB)

#############################
             resultado = self.partido(self.GrupoA.equipo2,self.GrupoA.equipo4)
             self.tabla.resultado(self.GrupoA.equipo2,self.GrupoA.equipo4,resultado.golesA,resultado.golesB)
             self.GrupoA.tabla.resultado(self.GrupoA.equipo2,self.GrupoA.equipo4,resultado.golesA,resultado.golesB)

             resultado = self.partido(self.GrupoB.equipo2,self.GrupoB.equipo4)
             self.tabla.resultado(self.GrupoB.equipo2,self.GrupoB.equipo4,resultado.golesA,resultado.golesB)
             self.GrupoB.tabla.resultado(self.GrupoB.equipo2,self.GrupoB.equipo4,resultado.golesA,resultado.golesB)

             resultado = self.partido(self.GrupoC.equipo2,self.GrupoC.equipo4)
             self.tabla.resultado(self.GrupoC.equipo2,self.GrupoC.equipo4,resultado.golesA,resultado.golesB)
             self.GrupoC.tabla.resultado(self.GrupoC.equipo2,self.GrupoC.equipo4,resultado.golesA,resultado.golesB)

             resultado = self.partido(self.GrupoD.equipo2,self.GrupoD.equipo4)
             self.tabla.resultado(self.GrupoD.equipo2,self.GrupoD.equipo4,resultado.golesA,resultado.golesB)
             self.GrupoD.tabla.resultado(self.GrupoD.equipo2,self.GrupoD.equipo4,resultado.golesA,resultado.golesB)
             
             resultado = self.partido(self.GrupoE.equipo2,self.GrupoE.equipo4)
             self.tabla.resultado(self.GrupoE.equipo2,self.GrupoE.equipo4,resultado.golesA,resultado.golesB)
             self.GrupoE.tabla.resultado(self.GrupoE.equipo2,self.GrupoE.equipo4,resultado.golesA,resultado.golesB)

             resultado = self.partido(self.GrupoF.equipo2,self.GrupoF.equipo4)
             self.tabla.resultado(self.GrupoF.equipo2,self.GrupoF.equipo4,resultado.golesA,resultado.golesB)
             self.GrupoF.tabla.resultado(self.GrupoF.equipo2,self.GrupoF.equipo4,resultado.golesA,resultado.golesB)
             
             resultado = self.partido(self.GrupoG.equipo2,self.GrupoG.equipo4)
             self.tabla.resultado(self.GrupoG.equipo2,self.GrupoG.equipo4,resultado.golesA,resultado.golesB)
             self.GrupoG.tabla.resultado(self.GrupoG.equipo2,self.GrupoG.equipo4,resultado.golesA,resultado.golesB)

             resultado = self.partido(self.GrupoH.equipo2,self.GrupoH.equipo4)
             self.tabla.resultado(self.GrupoH.equipo2,self.GrupoH.equipo4,resultado.golesA,resultado.golesB)
             self.GrupoH.tabla.resultado(self.GrupoH.equipo2,self.GrupoH.equipo4,resultado.golesA,resultado.golesB)
            
        if numero==3:
             resultado = self.partido(self.GrupoA.equipo1,self.GrupoA.equipo4)
             self.tabla.resultado(self.GrupoA.equipo1,self.GrupoA.equipo4,resultado.golesA,resultado.golesB)
             self.GrupoA.tabla.resultado(self.GrupoA.equipo1,self.GrupoA.equipo4,resultado.golesA,resultado.golesB)

             resultado = self.partido(self.GrupoB.equipo1,self.GrupoB.equipo4)
             self.tabla.resultado(self.GrupoB.equipo1,self.GrupoB.equipo4,resultado.golesA,resultado.golesB)
             self.GrupoB.tabla.resultado(self.GrupoB.equipo1,self.GrupoB.equipo4,resultado.golesA,resultado.golesB)

             resultado = self.partido(self.GrupoC.equipo1,self.GrupoC.equipo4)
             self.tabla.resultado(self.GrupoC.equipo1,self.GrupoC.equipo4,resultado.golesA,resultado.golesB)
             self.GrupoC.tabla.resultado(self.GrupoC.equipo1,self.GrupoC.equipo4,resultado.golesA,resultado.golesB)

             resultado = self.partido(self.GrupoD.equipo1,self.GrupoD.equipo4)
             self.tabla.resultado(self.GrupoD.equipo1,self.GrupoD.equipo4,resultado.golesA,resultado.golesB)
             self.GrupoD.tabla.resultado(self.GrupoD.equipo1,self.GrupoD.equipo4,resultado.golesA,resultado.golesB)
             
             resultado = self.partido(self.GrupoE.equipo1,self.GrupoE.equipo4)
             self.tabla.resultado(self.GrupoE.equipo1,self.GrupoE.equipo4,resultado.golesA,resultado.golesB)
             self.GrupoE.tabla.resultado(self.GrupoE.equipo1,self.GrupoE.equipo4,resultado.golesA,resultado.golesB)

             resultado = self.partido(self.GrupoF.equipo1,self.GrupoF.equipo4)
             self.tabla.resultado(self.GrupoF.equipo1,self.GrupoF.equipo4,resultado.golesA,resultado.golesB)
             self.GrupoF.tabla.resultado(self.GrupoF.equipo1,self.GrupoF.equipo4,resultado.golesA,resultado.golesB)
             
             resultado = self.partido(self.GrupoG.equipo1,self.GrupoG.equipo4)
             self.tabla.resultado(self.GrupoG.equipo1,self.GrupoG.equipo4,resultado.golesA,resultado.golesB)
             self.GrupoG.tabla.resultado(self.GrupoG.equipo1,self.GrupoG.equipo4,resultado.golesA,resultado.golesB)

             resultado = self.partido(self.GrupoH.equipo1,self.GrupoH.equipo4)
             self.tabla.resultado(self.GrupoH.equipo1,self.GrupoH.equipo4,resultado.golesA,resultado.golesB)
             self.GrupoH.tabla.resultado(self.GrupoH.equipo1,self.GrupoH.equipo4,resultado.golesA,resultado.golesB)

            
            
#########
             resultado = self.partido(self.GrupoA.equipo2,self.GrupoA.equipo3)
             self.tabla.resultado(self.GrupoA.equipo2,self.GrupoA.equipo3,resultado.golesA,resultado.golesB)
             self.GrupoA.tabla.resultado(self.GrupoA.equipo2,self.GrupoA.equipo3,resultado.golesA,resultado.golesB)

             resultado = self.partido(self.GrupoB.equipo2,self.GrupoB.equipo3)
             self.tabla.resultado(self.GrupoB.equipo2,self.GrupoB.equipo3,resultado.golesA,resultado.golesB)
             self.GrupoB.tabla.resultado(self.GrupoB.equipo2,self.GrupoB.equipo3,resultado.golesA,resultado.golesB)

             resultado = self.partido(self.GrupoC.equipo2,self.GrupoC.equipo3)
             self.tabla.resultado(self.GrupoC.equipo2,self.GrupoC.equipo3,resultado.golesA,resultado.golesB)
             self.GrupoC.tabla.resultado(self.GrupoC.equipo2,self.GrupoC.equipo3,resultado.golesA,resultado.golesB)

             resultado = self.partido(self.GrupoD.equipo2,self.GrupoD.equipo3)
             self.tabla.resultado(self.GrupoD.equipo2,self.GrupoD.equipo3,resultado.golesA,resultado.golesB)
             self.GrupoD.tabla.resultado(self.GrupoD.equipo2,self.GrupoD.equipo3,resultado.golesA,resultado.golesB)
             
             resultado = self.partido(self.GrupoE.equipo2,self.GrupoE.equipo3)
             self.tabla.resultado(self.GrupoE.equipo2,self.GrupoE.equipo3,resultado.golesA,resultado.golesB)
             self.GrupoE.tabla.resultado(self.GrupoE.equipo2,self.GrupoE.equipo3,resultado.golesA,resultado.golesB)

             resultado = self.partido(self.GrupoF.equipo2,self.GrupoF.equipo3)
             self.tabla.resultado(self.GrupoF.equipo2,self.GrupoF.equipo3,resultado.golesA,resultado.golesB)
             self.GrupoF.tabla.resultado(self.GrupoF.equipo2,self.GrupoF.equipo3,resultado.golesA,resultado.golesB)
             
             resultado = self.partido(self.GrupoG.equipo2,self.GrupoG.equipo3)
             self.tabla.resultado(self.GrupoG.equipo2,self.GrupoG.equipo3,resultado.golesA,resultado.golesB)
             self.GrupoG.tabla.resultado(self.GrupoG.equipo2,self.GrupoG.equipo3,resultado.golesA,resultado.golesB)

             resultado = self.partido(self.GrupoH.equipo2,self.GrupoH.equipo3)
             self.tabla.resultado(self.GrupoH.equipo2,self.GrupoH.equipo3,resultado.golesA,resultado.golesB)
             self.GrupoH.tabla.resultado(self.GrupoH.equipo2,self.GrupoH.equipo3,resultado.golesA,resultado.golesB)
            
            

    def partido(self,equipo1,equipo2):
        
        ###################################################################
        home=equipo1.nombre
        away=equipo2.nombre
        
        print("{} vs. {} : ".format(home,
                                   away), 
                                   end='')
        
        row = pd.DataFrame(np.array([[np.nan, np.nan, np.nan, True, np.nan, np.nan]])
                           , columns=X.columns)
        row['rank_home'] = world_cup_rankings.loc[home, 'rank']
        home_points = world_cup_rankings.loc[home, 'weighted_points']
        opp_rank = world_cup_rankings.loc[away, 'rank']
        opp_points = world_cup_rankings.loc[away, 'weighted_points']
        row['rank_difference'] = row['rank_home'] - opp_rank
        row['point_difference'] = home_points - opp_points
        row['Rating_home'] = world_cup_rankings.loc[home, 'Rating']
        opp_elo = world_cup_rankings.loc[away, 'Rating']
        opp_winptc = world_cup_rankings.loc[away, 'Win_pct']
        opp_avgrating = world_cup_rankings.loc[away, 'Avg Rating']
        opp_yearrating = world_cup_rankings.loc[away, 'Year Rating']
        row['elo_difference'] = world_cup_rankings.loc[home, 'Rating'] - opp_elo
        #row['winptc_difference'] = world_cup_rankings.loc[home, 'Win_pct'] - opp_winptc
        #row['avgrating_difference'] = world_cup_rankings.loc[home, 'Avg Rating'] - opp_avgrating
        #row['yearrating_difference'] = world_cup_rankings.loc[home, 'Year Rating'] - opp_yearrating
  

        #row_scaled = scaler.transform(row)
        #row_scaled = pd.DataFrame(row_scaled, columns=['rank_home', 'rank_difference', 'point_difference', 'is_stake'
                      #,'Rating_home','elo_difference','winptc_difference','yearrating_difference' ])
        #row['yearrating_difference'] = row_scaled['yearrating_difference']
        
        #print(row_scaled)
                
        #print(row)
        # valores de apuestas para ganador de mundial
        bet_prob_home = world_cup_rankings.loc[home, 'Prob'] 
        bet_prob_away = world_cup_rankings.loc[away, 'Prob'] 
        #dado que a menor numero, mejor, saco proporcion y si es menor al 25% de diferencia, asigno prob extra
        bet_prob_pct = bet_prob_home/(bet_prob_home + bet_prob_away)
        if bet_prob_pct < 0.25:
            bet_prob = 0.075
        elif bet_prob_pct >= 0.25 and bet_prob_pct < 0.40:
              bet_prob = 0.05
        else: bet_prob = 0
        home_win_prob = (model.predict_proba(row)[:,1][0]) + bet_prob
        # home_win_prob = model(row, batch_size=1)[0][0]
        print(home_win_prob)
        if abs(home_win_prob - 0.03) < 0.05:
            print("{0} Empate with probability {1:.2f}".format(home, home_win_prob))
            return Resultado(1,1)
#            winners.append(away)
        if (home_win_prob > .5):
            print("{0} equipo1 wins with probability {1:.2f}".format(home, home_win_prob))
            return Resultado(1,0)
        else:
            print("{0} equipo2 wins with probability {1:.2f}".format(away, 1-home_win_prob))
#            winners.append(home)
            return Resultado(0,1)
        
        
        
        
        
        
        
        
        #######################################################################
        
        
#        if abs(equipo1.ranking - equipo2.ranking) < .2:
#            return Resultado(1,1)
#        if equipo1.ranking > equipo2.ranking:
#            return Resultado(1,0)
#        if equipo1.ranking < equipo2.ranking:
#            return Resultado(0,1)
            
            
   
mundial = Mundial()
mundial.jugarFechaGrupo(1)
mundial.jugarFechaGrupo(2)
mundial.jugarFechaGrupo(3)

print(mundial.GrupoA.tabla)
print(mundial.GrupoB.tabla)
print(mundial.GrupoC.tabla)
print(mundial.GrupoD.tabla)
print(mundial.GrupoE.tabla)
print(mundial.GrupoF.tabla)
print(mundial.GrupoG.tabla)
print(mundial.GrupoH.tabla)

teamlist = (mundial.GrupoA.clasificados()+mundial.GrupoB.clasificados()+mundial.GrupoC.clasificados()+mundial.GrupoD.clasificados()+mundial.GrupoE.clasificados()+mundial.GrupoF.clasificados()+mundial.GrupoG.clasificados()+mundial.GrupoH.clasificados())
teamlistClasificados = teamlist
b = [0, 3, 4,7,8,11,12,15,2,1,6,5,10,9,14,13]
teamlist = [teamlist[i] for i in b]

teamlist[0].fase = Rondas.Octavos
for team in teamlist: mundial.tabla.setFase(team ,Rondas.Octavos)

## this list represents the round 1 seeding order on the tournament sheet
## this doesnt easily scale to 32, 64, 128 entries
#teamlist = [1,16,8,9,5,12,4,13,6,11,3,14,7,10,2,15]  

#In a single elim tournament with a full field, # of games is # of teams-1
totalgames = len(teamlist) - 1

#Set defaults
gameid = 0
roundid = 0
nextround = []
pasaA = Rondas.Cuartos
tPuesto =[]

#simulate all of the games
while gameid < totalgames:
    if gameid == 0: pasaA = Rondas.Cuartos
    if gameid == 8: pasaA = Rondas.Semifinal
    if gameid == 12: pasaA = Rondas.Final
    if gameid == 14: pasaA = Rondas.Campeon

    if gameid in [8,12,14,15]:  ##this is a manual decision tree, doesn't scale at all
        #if a new round begins, reset the list of the next round
        print("--- starting a new round of games ---Pasan a " + str(pasaA))
        print("next round matchup list: " + ', '.join([o.nombre for o in nextround]) + '\n\n')
        teamlist = nextround
        nextround = []
        roundid = 0

    #compare the 1st entry in the list to the 2nd entry in the list
    homeid = teamlist[roundid]
    awayid = teamlist[roundid + 1]

    #the winner of the match become the next entry in the nextround list
    #more realistic metrics could be substituted here, but ID can be used for this example

    resultado = mundial.partido(homeid,awayid)
    mundial.tabla.resultado(homeid,awayid,resultado.golesA,resultado.golesB)
    
    if (resultado.golesA > resultado.golesB):
        nextround.append(homeid)
        print(str(homeid) + " vs " + str(awayid) + ":  The winner is " + str(homeid))
        mundial.tabla.setFase(homeid ,pasaA)
        if (pasaA == Rondas.Final): tPuesto.append(awayid)

    else:
        nextround.append(awayid)
        print(str(homeid) + " vs " + str(awayid) + ":  The winner is " + str(awayid))
        mundial.tabla.setFase(awayid ,pasaA)
        if (pasaA == Rondas.Final): tPuesto.append(homeid)
        
    #increase the gameid and roundid
    gameid += 1
    roundid += 2

resultado = mundial.partido(tPuesto[0],tPuesto[1])
mundial.tabla.resultado(tPuesto[0],tPuesto[1],resultado.golesA,resultado.golesB)
if (resultado.golesA > resultado.golesB):
    mundial.tabla.setFase(tPuesto[0] ,Rondas.TercerPuesto)
else:
    mundial.tabla.setFase(tPuesto[1] ,Rondas.TercerPuesto)
    

print(mundial.tabla)



 



Russia vs. Saudi Arabia : 0.842773437009
Russia equipo1 wins with probability 0.84
Portugal vs. Spain : 0.26836650791
Spain equipo2 wins with probability 0.73
France vs. Australia : 0.80490030157
France equipo1 wins with probability 0.80
Argentina vs. Iceland : 0.80490030157
Argentina equipo1 wins with probability 0.80
Brazil vs. Switzerland : 0.839962264701
Brazil equipo1 wins with probability 0.84
Germany vs. Mexico : 0.82698909944
Germany equipo1 wins with probability 0.83
Belgium vs. Panama : 0.885396636674
Belgium equipo1 wins with probability 0.89
Poland vs. Senegal : 0.482619765217
Senegal equipo2 wins with probability 0.52
Egypt vs. Uruguay : 0.0867491198855
Uruguay equipo2 wins with probability 0.91
Morocco vs. Iran : 0.361083218609
Iran equipo2 wins with probability 0.64
Peru vs. Denmark : 0.42331947491
Denmark equipo2 wins with probability 0.58
Croatia vs. Nigeria : 0.702354897806
Croatia equipo1 wins with probability 0.70
Costa Rica vs. Serbia : 0.3140870999
Serbia equipo2 